## Multi-fidelity Bayesian optimization (MFBO) of COFs for Xe/Kr separations.
1. We have a set of COFs from a database. Each COF is characterized by a feature vector $$x \in \mathcal{X} \subset R^d$$ were d=14.

2. We have **two different types** of simulations to calculate **the same material property**, the adsorptive Xe/Kr selectivity $S_{Xe/Kr}$. However, we only have a single objective: to maximize the high-fidelity selectivity. 
$$\arg\max_{x \in \mathcal{X}}[S^{(\ell=\text{high})}_{Xe/Kr}(x)]$$

3. Multi-Fidelity options: 
    1. low-fidelity  => Henry coefficient calculation - MC integration: $S_{Xe/Kr}^{\text{low}} = \frac{H_{Xe}}{H_{Kr}}$
    2. high-fidelity => GCMC mixture simulation - 80:20 (Kr:Xe) at 298 K and 1.0 bar: $S_{Xe/Kr}^{\text{high}} = \frac{n_{Xe} / n_{Kr}}{y_{Xe}/y_{Kr}}$


3. We will initialize the surrogate model with a few (3) COFs with simulations under **both** fidelities.
    1. The fist COF will be the one closest to the center of the normalized feature space
    2. The rest will be chosen to maximize diversity of the training set


4. Model:
    1. Botorch GP surrogate model: [SingleTaskMultiFidelityGP](https://botorch.org/api/models.html#module-botorch.models.gp_regression_fidelity) or [FixedNoiseMultiFidelityGP](https://botorch.org/api/models.html#botorch.models.gp_regression_fidelity.FixedNoiseMultiFidelityGP)
        - Needed to use [this](https://botorch.org/api/optim.html#module-botorch.optim.fit) optimizer to correct matrix jitter
    2. We  use the augmented-EI (aEI) acquisition function from [here](https://link.springer.com/content/pdf/10.1007/s00158-005-0587-0.pdf)


-  Helpful [tutorial](https://botorch.org/tutorials/discrete_multi_fidelity_bo) for a similar BoTorch Model used

In [1]:
import torch
import gpytorch

from botorch.models import SingleTaskMultiFidelityGP

from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
###
#  fix Cholecky jitter error:
#  the line search in the L-BFGS algorithm, used by default, can take some very large steps, 
#  which in turn causes numerical issues in the solves in the underlying gpytorch model.
#
#  recommended solution: use an optimizer from torch.optim to fit the model
#  see [posted issue here](https://github.com/pytorch/botorch/issues/179#issuecomment-504798767)
###
from botorch.optim.fit import fit_gpytorch_torch 

from scipy.stats import norm
import numpy as np
import pickle
import h5py # for .jld2 files
import os
import time

import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_absolute_error

/Users/cokes/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
###
#  figure settings 
###
import seaborn as sns
sns.set(style='ticks', palette='Set2', font_scale=1.5, rc={"lines.linewidth": 3})
sns.despine()
# plt.rcParams.update({'font.size': 16})
# plt.rcParams['figure.dpi'] = 1200 # 600-1200 for paper quality

save_figures = True

<Figure size 640x480 with 0 Axes>

## Load Data

In [4]:
ablation_study_flag = False # make features have no information for a baseline. to gauge feature importance

first, the targets (simulated adsorption) and features of the COFs.

In [47]:
file = h5py.File("./targets_and_raw_features.jld2", "r")

xtal_names = file['COFs'][:]

feature_names = file['feature_names'][:]
feature_names = [fn.decode() for fn in feature_names]

# feature matrix
X = torch.from_numpy(np.transpose(file["X"][:]))

if ablation_study_flag:
    # shuffle columns
    for j in range(X.size()[1]):
        shuffled_row_ids = torch.randperm(X.size()[0])
        X[:, j] = X[shuffled_row_ids, j]

# simulation data
y = [torch.from_numpy(np.transpose(file["henry_y"][:])), 
     torch.from_numpy(np.transpose(file["gcmc_y"][:]))]  

# associated simulation costs
cost = [np.transpose(file["henry_total_elapsed_time"][:]), # [min]
        np.transpose(file["gcmc_elapsed_time"][:])]        # [min]

# total number of COFs in data set
nb_COFs = X.shape[0]

second, the COFs to initialize the surrogate model.

In [12]:
init_cof_ids_file = pickle.load(open('../search_results/initializing_cof_ids_normalized.pkl', 'rb'))

init_cof_ids = init_cof_ids_file['init_cof_ids']

# total number of BO searches to run = number of initializing sets
nb_runs = len(init_cof_ids)
nb_runs

100

some tests on the input data.

In [25]:
for i, f in enumerate(feature_names):
    print("{}: {}".format(i, f))

0: pore_diameter_Å
1: void_fraction
2: surface_area_m²g⁻¹
3: crystal_density
4: B
5: O
6: C
7: H
8: Si
9: N
10: S
11: P
12: halogens
13: metals


In [53]:
rnd_COF = b'19060N2_ddec.cif' # random COF

id_rnd_COF = np.where(xtal_names == rnd_COF)[0]

# does the low-fidelity selectivity match that manually read from the simulation output file?
assert np.isclose(y[0][id_rnd_COF].item(), 722.409 / 202.085)

# does the high-fidelity selectivity match that manually read from the simulation output file?
assert np.isclose(y[1][id_rnd_COF].item(), (6.1558810248879325 / 6.842906773660418) / (20/80))

# manually check some features
if not ablation_study_flag:
    # ASA_m^2/g = 4363.81 in Zeo++ output file. this better match!
    assert np.isclose(X[id_rnd_COF, 2].item(), 4363.81)

    # mol fraction of N = 0.04807692307692308 in the xtal. this better match!
    assert np.isclose(X[id_rnd_COF, 9].item(), 0.04807692307692308)
    
    # sum of mol frac's = 1
    assert X[id_rnd_COF, 4:].sum().item() == 1
    
    # pore diameter = 15.12574 from Zeo++ output file
    assert X[id_rnd_COF, 0] == 15.12574
    
    # void fraction = 0.58554 from Zeo++ output file.
    assert X[id_rnd_COF, 1] == 0.58554
    
    # xtal density from Zeo++ output = 0.604869
    assert np.isclose(X[id_rnd_COF, 3], 0.604869 * 1000, atol=0.1) # unit convert cuz computed in PM.jl

now that we've tested, normalize the features to lie in [0, 1]

In [54]:
for j in range(X.size()[1]):
    X[:, j] = (X[:, j] - torch.min(X[:, j]).item()) / (torch.max(X[:, j]).item() - torch.min(X[:, j]).item())

print stuff

In [55]:
# cost
print("total high-fidelity cost:", sum(cost[1]).item(), "[min]")
print("total low-fidelity cost: ", sum(cost[0]).item(), "[min]")
print("average high-fidelity cost:", np.mean(cost[1]), "[min]")
print("average low-fidelity cost: ", np.mean(cost[0]), "[min]")
print("average cost ratio:\t   ", np.mean(cost[1] / cost[0]))

# data shape
print("\nraw data - \n\tX:", X.shape)
for f in range(2):
    print("\tfidelity:", f)
    print("\t\ty:", y[f].shape)
    print("\t\tcost: ", cost[f].shape)
    
# normalization check
print("\nEnsure features are normalized - ")
print("max:\n", torch.max(X, 0).values)
print("min:\n", torch.min(X, 0).values)
print("width:\n",torch.max(X, 0).values - torch.min(X, 0).values)
print("mean:\n", torch.mean(X, 0))
print("std:\n", torch.std(X, 0))

total high-fidelity cost: 139887.66223703226 [min]
total low-fidelity cost:  10076.305239888028 [min]
average high-fidelity cost: 230.0783918372241 [min]
average low-fidelity cost:  16.57287046034216 [min]
average cost ratio:	    13.444745568580501

raw data - 
	X: torch.Size([608, 14])
	fidelity: 0
		y: torch.Size([608])
		cost:  (608,)
	fidelity: 1
		y: torch.Size([608])
		cost:  (608,)

Ensure features are normalized - 
max:
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64)
min:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)
width:
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64)
mean:
 tensor([0.2856, 0.5864, 0.5304, 0.3323, 0.0421, 0.1617, 0.6405, 0.6793, 0.0062,
        0.1758, 0.0308, 0.0131, 0.0132, 0.0238], dtype=torch.float64)
std:
 tensor([0.1586, 0.1787, 0.1896, 0.1512, 0.1150, 0.1916, 0.1631, 0.1354, 0.0631,
        0.1126, 0.1026, 0.0935,

## Helper Functions

#### Post-Search Analysis

In [64]:
def calc_fidelity_fraction(acquired_set, fidelity):
    assert fidelity in [1/3, 2/3] 
    nb_iters = len(acquired_set)
    fid_frac = np.zeros(nb_iters)
    for i in range(1, nb_iters):
        fid_frac[i] = sum(acquired_set[:, 0][:i+1] == fidelity) / (i+1)
    return fid_frac

In [65]:
# very crude way to get fidelity id
# exploits lf < 0.5 and hf > 0.5
# reurns a 0 or 1
def get_f_ids(acquired_set):
    if acquired_set.dim() == 0:
        return acquired_set.round().to(dtype=int)
    else: 
        f_ids = [a[0].round().to(dtype=int) for a in acquired_set]
        return torch.tensor(f_ids)

In [66]:
def get_y_maxes_acquired(acquired_set):    
    nb_iters = len(acquired_set)
    y_maxes = np.zeros(nb_iters)
    # we want the maximum y value (only high-fidelity) up to a given iteration
    y_max = 0.0 # update this each iteration.
    for i, (f_val, cof_id) in enumerate(acquired_set):
        f_id = get_f_ids(torch.tensor(f_val))
        assert f_id in [0, 1]
        y_acq_this_iter = y[f_id][int(cof_id)]
        # i is iteration index
        if f_id == 1 and y_acq_this_iter > y_max:  
            y_max = y_acq_this_iter # over-write max
        y_maxes[i] = y_max 
    return y_maxes

In [67]:
def accumulated_cost(cost_acquired):
    nb_iters = len(acquired_set)
    accumulated_cost = np.zeros(nb_iters)
    accumulated_cost[0] = cost_acquired[0]
    for i in range(1, len(cost_acquired)):
        accumulated_cost[i] = accumulated_cost[i-1] + cost_acquired[i]
    return accumulated_cost

#### Construct Initial Inputs

In [68]:
def initialize_acquired_set(X, y, initializing_COFs, discrete_fidelities):
#     cof_ids = diverse_set(X, nb_COFs_initialization) # np.array(ids_train)
    return torch.tensor([[f_id, cof_id] for cof_id in initializing_COFs for f_id in discrete_fidelities])

In [69]:
# construct feature matrix of acquired points
def build_X_train(acquired_set):
    cof_ids = [a[1].to(dtype=int) for a in acquired_set]
    f_ids = torch.tensor([a[0] for a in acquired_set])
    assert f_ids[0] in [1/3, 2/3]
    return torch.cat((X[cof_ids, :], f_ids.unsqueeze(dim=-1)), dim=1)

# construct output vector for acquired points
def build_y_train(acquired_set):
    f_ids = get_f_ids(acquired_set)
    cof_ids = [a[1].to(dtype=int) for a in acquired_set]
    return torch.tensor([y[f_id][cof_id] for f_id, cof_id in zip(f_ids, cof_ids)]).unsqueeze(-1)

# construct vector to track accumulated cost of acquired points
def build_cost(acquired_set):
    f_ids = get_f_ids(acquired_set)
    cof_ids = [a[1].to(dtype=int) for a in acquired_set]
    return torch.tensor([cost[f_id][cof_id] for f_id, cof_id in zip(f_ids, cof_ids)]).unsqueeze(-1)

# construct vector to track accumulated cost of acquired points
def build_cost_fidelity(acquired_set, fidelity):
    f_ids = get_f_ids(acquired_set)
    cof_ids = [a[1].to(dtype=int) for a in acquired_set]
    return torch.tensor([cost[f_id][cof_id] for f_id, cof_id in zip(f_ids, cof_ids) if f_id == fidelity]).unsqueeze(-1)

some tests

In [88]:
np.allclose(X[[10, 3, 4], :], bogus_X_train[:, :14])

True

True

In [93]:
bogus_acquired_set = torch.tensor([[2/3, 10], [1/3, 3], [1/3, 4]])

###
#   build_X_train
###
bogus_X_train = build_X_train(bogus_acquired_set)
# first 14 are features.
np.allclose(X[[10, 3, 4], :], bogus_X_train[:, :14])

# last one is fidelity param
assert np.allclose(bogus_X_train[:, 14], [2/3, 1/3, 1/3])

###
#   build_y_train
###
bogus_y_train = build_y_train(bogus_acquired_set)
assert bogus_y_train[0] == y[1][10] # y[fid_id][cof_id]
assert bogus_y_train[1] == y[0][3]
assert bogus_y_train[2] == y[0][4]
#bogus_X_train[0, :14]

In [77]:
    assert y_train[0] == y[1][10] # y[fid_id][cof_id]
    assert y_train[2] == y[0][4]

tensor([[0.0241, 0.0959, 0.4220, 0.6899, 0.2895, 0.4211, 0.2811, 0.8421, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6667],
        [0.0815, 0.2693, 0.1253, 0.6615, 0.3929, 0.5714, 0.7213, 0.4286, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3333],
        [0.2144, 0.5197, 0.3039, 0.3981, 0.2292, 0.3333, 0.7561, 0.5833, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3333]], dtype=torch.float64)

In [71]:
def test_initializing_functions(X, y):
    ###
    #  construct training sets
    ###
    # list of (cof_id, fid_id)'s
    
    # Training Sets
    X_train = build_X_train(acquired_set)
    y_train = build_y_train(acquired_set)
    
    ###
    #  test that the constructor functions are working properly
    ###
    assert np.allclose(X[10, :], X_train[0, :14])
    assert np.isclose(X_train[0, 14], 2/3) # high-fidelity
    assert np.isclose(X_train[1, 14], 1/3) # low-fidelity
    assert y_train[0] == y[1][10] # y[fid_id][cof_id]
    assert y_train[2] == y[0][4]
    return

test_initializing_functions(X, y)

NameError: name 'acquired_set' is not defined

### Surrogate Model

In [ ]:
def train_surrogate_model(X_train, y_train):
    model = SingleTaskMultiFidelityGP(
        X_train, 
        y_train, 
        linear_truncated=False, # RBF for features and Downsampling for Fidelities
        outcome_transform=Standardize(m=1), # m is the output dimension
        data_fidelity=X_train.shape[1] - 1
    )   
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll, optimizer=fit_gpytorch_torch)
    return model

### Acquisition Function

In [ ]:
# calculate posterior mean and variance for a given fidelity
def mu_sigma(model, X, fidelity):
    assert fidelity in [1/3, 2/3]
    nb_COFs_here = X.size()[0]
    f = torch.tensor((), dtype=torch.float64).new_ones((nb_COFs_here, 1)) * fidelity
    X_f = torch.cat((X, f), dim=1) # last col is associated fidelity
    f_posterior = model.posterior(X_f)
    return f_posterior.mean.squeeze().detach().numpy(), np.sqrt(f_posterior.variance.squeeze().detach().numpy())

# get the current best y-value of desired_fidelity in the acquired set
def get_y_max(acquired_set, desired_fidelity):
    assert desired_fidelity in [0, 1]
    f_ids = get_f_ids(acquired_set)
    cof_ids = [a[1].to(dtype=int) for a in acquired_set]
    return np.max([y[f_id][cof_id] for f_id, cof_id in zip(f_ids, cof_ids) if f_id == desired_fidelity])

In [ ]:
###
#  efficient multi-fidelity correlation function
#  corr(y at given fidelity, y at high-fidelity)
#  (see notes)
###
def mfbo_correlation_function(model, X, fidelity):
    assert fidelity in [1/3, 2/3]
    # given fidelity
    f   = torch.tensor((), dtype=torch.float64).new_ones((nb_COFs, 1)) * fidelity
    X_f = torch.cat((X, f), dim=1) # last col is associated fidelity
    
    #  high-fidelity
    hf   = torch.tensor((), dtype=torch.float64).new_ones((nb_COFs, 1)) * discrete_fidelities[-1]
    X_hf = torch.cat((X, hf), dim=1) # last col is associated fidelity

    # combine into a single tensor
    X_all_fid = torch.cat((X_f, X_hf), dim=0)
    
    # get variance for each fidelity
    var_f = torch.flatten(model.posterior(X_f).variance)
    var_hf = torch.flatten(model.posterior(X_hf).variance) # variance
    
    # posterior covariance 
    cov = torch.diag(model(X_all_fid).covariance_matrix[:X_f.size()[0], X_f.size()[0]:])
    
    corr = cov / (torch.sqrt(var_f) * torch.sqrt(var_hf))
    return corr

In [ ]:
###
#  cost ratio
###
def estimate_cost_ratio(fidelity, acquired_set):
    assert fidelity in [1/3, 2/3]
    f_id = get_f_ids(torch.tensor(fidelity))
    avg_cost_f  = torch.mean(build_cost_fidelity(acquired_set, f_id))
    avg_cost_hf = torch.mean(build_cost_fidelity(acquired_set, 1))
    cr = avg_cost_hf / avg_cost_f
    return cr.item()

###
#  expected imrovement function, only uses hf
###
def EI_hf(model, X, acquired_set):
    hf_mu, hf_sigma = mu_sigma(model, X, discrete_fidelities[-1])
    y_max = get_y_max(acquired_set, 1)
    
    z = (hf_mu - y_max) / hf_sigma
    explore_term = hf_sigma * norm.pdf(z) 
    exploit_term = (hf_mu - y_max) * norm.cdf(z) 
    ei = explore_term + exploit_term
    return np.maximum(ei, np.zeros(nb_COFs))

###
#  acquisition function
###
def acquisition_scores(model, X, fidelity, acquired_set):
    assert fidelity in [1/3, 2/3]
    # expected improvement for high-fidelity
    ei = EI_hf(model, X, acquired_set) 
    
    # augmenting functions
    corr_f1_f0 = mfbo_correlation_function(model, X, fidelity)
    
    cr = estimate_cost_ratio(fidelity, acquired_set)

    scores = torch.from_numpy(ei) * corr_f1_f0 * cr
    return scores.detach().numpy()

def in_acquired_set(f_id, cof_id, acquired_set):
    assert f_id in [0, 1]
    fidelity = discrete_fidelities[f_id]
    for this_fidelity, this_cof_id in acquired_set:
        if this_cof_id == cof_id and this_fidelity == fidelity:
            return True
    return False

### Bayesian Algorithm

In [ ]:
def run_Bayesian_optimization(nb_iterations, initializing_COFs, verbose=False):
    assert nb_iterations > len(initializing_COFs)
    ###
    #  initialize system
    ###
    acquired_set = initialize_acquired_set(X, y, initializing_COFs, discrete_fidelities)
    
    ###
    #  itterate through remaining budget using BO
    ###
    for i in range(nb_COFs_initialization * len(discrete_fidelities), nb_iterations): 
        print("BO iteration: ", i)
        ###
        #  construct training data (perform experiments)
        ###
        X_train = build_X_train(acquired_set)
        y_train = build_y_train(acquired_set)

        if verbose:
            print("Initialization - \n")
            print("\tid acquired = ", [acq_[1].item() for acq_ in acquired_set])
            print("\tfidelity acquired = ", [acq_[0].item() for acq_ in acquired_set])
            print("\tcosts acquired = ", build_cost(acquired_set), " [min]")

            print("\n\tTraining data:\n")
            print("\t\t X train shape = ", X_train.shape)
            print("\t\t y train shape = ", y_train.shape)
            print("\t\t training feature vector = \n", X_train)
        
        ###
        #  train Model
        ###
        model = train_surrogate_model(X_train, y_train)
        
        ###
        #  acquire new (COF, fidelity) not yet acquired.
        ###
        # entry (fid_id, cof_id) is the acquisition value for fidelity f_id and cof cof_id
        the_acquisition_scores = np.array([acquisition_scores(model, X, fidelity, acquired_set) for fidelity in discrete_fidelities])
        
        # overwrite acquired COFs/fidelities with negative infinity to not choose these.
        for fidelity, cof_id in acquired_set:
            the_acquisition_scores[get_f_ids(fidelity), cof_id.to(dtype=int)] = - np.inf
        
        # select COF/fidelity with highest aquisition score.
        f_id, cof_id = np.unravel_index(np.argmax(the_acquisition_scores), np.shape(the_acquisition_scores))
        assert f_id in [0, 1]
        assert not in_acquired_set(f_id, cof_id, acquired_set)
        
        # update acquired_set
        acq = torch.tensor([[discrete_fidelities[f_id], cof_id]]) # dtype=int
        print(acq)
        acquired_set = torch.cat((acquired_set, acq))

        ###
        #  print useful info
        ###
        if verbose:
            print("\tacquired COF ", cof_id, " at fidelity, ", f_id)
            print("\t\ty = ", y[f_id][cof_id].item())
            print("\t\tcost = ", cost[f_id][cof_id])
        
    return acquired_set

# Run MFBO

In [ ]:
###
#  construct initial inputs
###
discrete_fidelities = [1/3, 2/3] # set of discrete fidelities (in ascending order) to select from

nb_COFs_initialization = 3   # at each fidelity, number of COFs to initialize with
nb_iterations = 150          # BO budget, includes initializing COFs


if ablation_study_flag:
    print("ablation study: {}".format(ablation_study_flag))
    # the maximum possible number itterations = num_fidelities * nb_COFs
    # this would efectively constitute a low-fidelity exhaustive search 
    # followed by a high-fidelity exhaustive search
    nb_iterations = 2 * nb_COFs 
    print("max. number of itterations: {}".format(nb_iterations))

**Using the set with COF closest to the center of feature space**

In [ ]:
ids_acquired = run_Bayesian_optimization(nb_iterations, init_cof_ids[0])

In [ ]:
# unpack search results
fids = [ids_acquired[i][0].item() for i in range(len(ids_acquired))]
cof_ids = [int(ids_acquired[i][1].item()) for i in range(len(ids_acquired))]

# which COF has the largest high-fidelity selectivity?
cof_id_with_max_hi_fid_selectivity = np.argmax(y[1]).item()

# iteration we found top COF
n_iter_top_cof_found = np.where([cof_ids[i] == cof_id_with_max_hi_fid_selectivity and fids[i] > 0.5 for i in range(len(cof_ids))])[0].item()
n_iter_top_cof_found

### observe status of surrogate model's knowledge the iteration before the top COF was acquired.

In [ ]:
# plot the selectivities of COFs according to low vs high fid
# which COFs are sampled at both fidelities?
hi_fid_cofs = [cof_ids[i] for i in range(n_iter_top_cof_found) if fids[i] > 0.5]
lo_fid_cofs = [cof_ids[i] for i in range(n_iter_top_cof_found) if fids[i] < 0.5]
ids_cofs_hi_and_lo_fid = np.intersect1d(hi_fid_cofs, lo_fid_cofs)
ids_cofs_hi_and_lo_fid

In [ ]:
# build selectivity array for plotting
y_los = [y[0][c].item() for c in ids_cofs_hi_and_lo_fid]
y_his = [y[1][c].item() for c in ids_cofs_hi_and_lo_fid]

fig = plt.figure()
plt.plot([0, 25], [0, 25], linestyle="--", color="k", linewidth=1)
plt.scatter(y_los, y_his, zorder=10)
ax = plt.gca()
plt.xlim(0, 25)
plt.ylim(0, 25)
ax.set_aspect("equal", "box")
plt.xlabel("low-fidelity Xe/Kr selectivity")
plt.ylabel("high-fidelity Xe/Kr selectivity")
plt.tight_layout()
if save_figures:
    plt.savefig("../figs/lo_vs_hi_fid_selectivity.pdf", format='pdf')
    
plt.show()

In [ ]:
# get COF ids not in acquired set.
test_cof_ids = [cof_id for cof_id in range(nb_COFs) if not (cof_id in hi_fid_cofs)]
len(test_cof_ids)

In [ ]:
cof_id_with_max_hi_fid_selectivity in test_cof_ids

In [ ]:
id_in_test_cofs_of_opt_cof = np.where([c == cof_id_with_max_hi_fid_selectivity for c in test_cof_ids])[0].item()
id_in_test_cofs_of_opt_cof

In [ ]:
# train surrogate model for test data, on acquired set up till top COF was found.
X_train = build_X_train(ids_acquired[:n_iter_top_cof_found])
y_train = build_y_train(ids_acquired[:n_iter_top_cof_found])

model = train_surrogate_model(X_train, y_train)

# get model predictions on test COFs, for high-fidelity.
y_pred, sigma = mu_sigma(model, X[test_cof_ids, :], discrete_fidelities[-1])

# plot true vs predicted
y_true = [y[1][c].item() for c in test_cof_ids]

r2 = r2_score(y_true, y_pred)
abserr = mean_absolute_error(y_true, y_pred)


###
#  parity plot
###
gridspec_kw={'width_ratios': [6, 2], 'height_ratios': [2, 6]} # set ratios
fig, ax = plt.subplots(nrows=2, ncols=2, gridspec_kw=gridspec_kw, figsize=(8, 8))
# fig = plt.figure()
ax[0, 1].axis("off")
ax[1, 0].plot([0, 20], [0, 20], linestyle="--", color="k", linewidth=1)
# ax = plt.gca()
ax[1, 0].set_xlim(0, 20)
ax[1, 0].set_ylim(0, 20)

#ax[1, 0].set_aspect("equal", "box")

ax[1, 0].text(5, 15, "R$^2$=%.2f\nMAE=%.2f" % (r2, abserr))
ax[1, 0].scatter(y_true, y_pred, fc='none', ec="k")
ax[1, 0].set_xlabel("true\nhigh-fidelity Xe/Kr selectivity")
ax[1, 0].set_ylabel("predicted\nhigh-fidelity Xe/Kr selectivity")
# plot acquired COF
ax[1, 0].scatter(y_true[id_in_test_cofs_of_opt_cof], y_pred[id_in_test_cofs_of_opt_cof], marker="x", color="red")


###
#  histogram of selectivities
###
hist_color = sns.color_palette("husl", 8)[7]
ax[0, 0].hist(y_true, color=hist_color, alpha=0.5) # 
ax[0, 0].sharex(ax[1, 0])
ax[0, 0].set_ylabel('# COFs')
plt.setp(ax[0, 0].get_xticklabels(), visible=False) # remove yticklabels

hist_color = sns.color_palette("husl", 8)[7]
ax[1, 1].hist(y_pred, color=hist_color, alpha=0.5, orientation="horizontal") # 
ax[1, 1].sharey(ax[1, 0])
ax[1, 1].set_xlabel('# COFs')
plt.setp(ax[1, 1].get_yticklabels(), visible=False) # remove yticklabels

sns.despine()
plt.tight_layout()
if save_figures:
    plt.savefig("../figs/surrogate_parity_with_hist.pdf", format="pdf")

plt.show()

In [ ]:
ids_sorted = np.argsort(y_true)[::-1]

plt.figure(figsize=(12, 3))
plt.errorbar(range(len(y_true)), y_pred[ids_sorted], yerr=sigma, linewidth=1, marker="o")
plt.errorbar(0, y_pred[id_in_test_cofs_of_opt_cof], yerr=sigma[id_in_test_cofs_of_opt_cof], linewidth=1, marker="o", color="red")
plt.xlabel("rank according to true high-fidelity Xe/Kr selectivity")
plt.ylabel("predicted\nhigh-fidelity\nXe/Kr selectivity")
plt.xlim(-1, len(test_cof_ids) +1)
plt.tight_layout()
if save_figures:
    plt.savefig("../figs/surrogate_predictions.pdf", format="pdf")

plt.show()

In [ ]:
cof_id_with_max_hi_fid_selectivity

In [ ]:
np.where([cof_id == cof_id_with_max_hi_fid_selectivity for cof_id in test_cof_ids])

# Run MFBO under different initializations

In [ ]:
###
#  run search
###
for j, initializing_COFs in enumerate(init_cof_ids): 

    # each time, randomly shuffle features (with no info in them about the COFs)
    if ablation_study_flag:
        for j in range(X.size()[1]):
            row_ids = torch.randperm(X.size()[0])
            X[:, j] = X[row_ids, j]

            
    # check the length of each initializing set
    assert len(initializing_COFs) == nb_COFs_initialization
    print("run #: {}".format(j))
    
    # start timer for BO run
    start_time = time.time()
    ###
    #  run BO search
    ###
    acquired_set = run_Bayesian_optimization(nb_iterations, initializing_COFs)
    
    ###
    #  post-run analysis
    ###
    elapsed_time  = time.time() - start_time
    print("elapsed_time:\t", elapsed_time / 60, " min.")
    y_acquired    = build_y_train(acquired_set)
    y_maxes_acq   = get_y_maxes_acquired(acquired_set.detach().numpy())
    fid_fraction  = calc_fidelity_fraction(acquired_set.detach().numpy(), discrete_fidelities[1])
    cost_acquired = build_cost(acquired_set)
    acc_cost      = accumulated_cost(cost_acquired.flatten().detach().numpy())
    
    ###
    # look at unique COFs acquired
    ###
    # cof_ids_acquired = torch.tensor([acq[1] for acq in acquired_set])
    n_unique_cofs_acquired = len(np.unique([acq[1] for acq in acquired_set]))
    print("total number of unique COFs acquired", n_unique_cofs_acquired)

    ###
    #  Iterations until top COF and accumulated 
    ###
    cof_id_with_max_selectivity = np.argmax(y[1])
    BO_iter_top_cof_acquired = float("inf") # dummy 
    for i, (f_id, cof_id) in enumerate(acquired_set):
        if cof_id.to(dtype=int) == cof_id_with_max_selectivity and get_f_ids(f_id) == 1:
            BO_iter_top_cof_acquired = i
            print("woo, top COF acquired!")
            print("iteration we acquire top COF = ", BO_iter_top_cof_acquired) 
            break
        elif i == len(acquired_set)-1:
            print("oh no, top COF not acquired!")

    # print accumulated cost to find the optimal COF
    top_cof_acc_cost = sum(build_cost(acquired_set)[:BO_iter_top_cof_acquired])
    print("accumulated cost up to observation of top COF = ", top_cof_acc_cost.item(), " [min]")
    
    ###
    #  store results
    ###
    mfbo_res = dict({'acquired_set': acquired_set.detach().numpy(),
                     'y_acquired': y_acquired.detach().numpy(),
                     'y_max_acquired': y_maxes_acq,
                     'fidelity_fraction': fid_fraction,
                     'cost_acquired': cost_acquired.flatten().detach().numpy(),
                     'accumulated_cost': acc_cost / 60,
                     'nb_COFs_initialization': nb_COFs_initialization,
                     'BO_iter_top_cof_acquired': BO_iter_top_cof_acquired,
                     'elapsed_time (min)':  elapsed_time / 60
                    })
    pickle_filename = '../search_results/mfbo_results/mfbo_results_run_{}'.format(j)
    if ablation_study_flag:
        pickle_filename += "_ablation"
    pickle_filename += ".pkl"
    with open(pickle_filename, 'wb') as file:
        pickle.dump(mfbo_res, file)